In [1]:
# style파일 apr.doc.print.jstl.css만 사용
# <!DOCTYPE html><html lang="en"><head><body>태그 제거
# .content_box{min-width:850->819px로 변경, width:99.9% 삭제
# ul,ol{list-style:none !important; padding-left: 0 !important; margin-left: 0 !important;} 으로 변경
# .content_box{width:97.7% !important;}삭제

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
 [기능]
 HTML 파일에 CSS를 인라인 삽입하고 압축하여 DB에 저장

 [처리 순서]
 1. CSS 파일 읽기 (apr.doc.print.jstl.css만 사용)
 2. CSS 규칙 수정
    - .content_box: min-width 850px→819px, width:99.9% 삭제
    - ul,ol: list-style:none !important + padding/margin 0
 3. HTML의 <link> 태그를 <style> 태그로 교체 (CSS 인라인화)
 4. 불필요한 섹션 삭제
    - 첨부파일, 참조문서, 결재의견, 결재댓글 영역
 5. 커스텀 CSS 추가 (.content, .content table 너비 조정)
 6. <style> 태그를 </body> 직전으로 이동
 7. HTML wrapper 태그 제거
    - <!DOCTYPE html>, <html>, <head>, <body> 등
 8. HTML/CSS 압축 (한 줄로 minify)
 9. DB의 doc_body 컬럼에 UPDATE

 [출력]
 - MariaDB: documents 테이블의 doc_body 컬럼 업데이트
 - 실패 시: failed_updates.txt 파일 생성

 [설정 (#여기를 수정하세요)]
 - base_path: HTML 폴더 경로
 - end_year: 기간 구분용 연도
 - db_config: DB 접속 정보 (host, user, password, database)

 [의존성]
 - beautifulsoup4
 - pymysql
-> pip install beautifulsoup4 pymysql lxml
"""
import json
import re
from pathlib import Path
from bs4 import BeautifulSoup
import pymysql

def minify_css(css):
    """CSS를 한 줄로 압축 (렌더링 결과는 동일)"""
    # CSS 주석 제거
    css = re.sub(r'/\*.*?\*/', '', css, flags=re.DOTALL)
    # 줄바꿈 제거
    css = css.replace('\n', '').replace('\r', '')
    # 연속된 공백을 하나로
    css = re.sub(r'\s+', ' ', css)
    # { } : ; 앞뒤 공백 제거
    css = re.sub(r'\s*{\s*', '{', css)
    css = re.sub(r'\s*}\s*', '}', css)
    css = re.sub(r'\s*:\s*', ':', css)
    css = re.sub(r'\s*;\s*', ';', css)
    css = re.sub(r'\s*,\s*', ',', css)
    return css.strip()

def minify_html(html):
    """
    HTML을 완전히 한 줄로 압축 (CSS 포함)
    - <pre>, <textarea> 태그 내부만 보존
    - CSS, Script도 모두 한 줄로 압축
    """
    # 1. <style> 태그 내부 CSS를 압축
    def minify_style_tag(match):
        tag_open = match.group(1)  # <style...>
        css_content = match.group(2)
        minified_css = minify_css(css_content)
        return f"{tag_open}{minified_css}</style>"
    
    html = re.sub(r'(<style[^>]*>)(.*?)</style>', minify_style_tag, html, flags=re.DOTALL | re.IGNORECASE)
    
    # 2. <script> 태그 내부를 한 줄로 압축
    def minify_script_tag(match):
        tag_open = match.group(1)
        script_content = match.group(2)
        # 스크립트는 간단히 줄바꿈과 연속 공백만 제거
        minified = script_content.replace('\n', '').replace('\r', '')
        minified = re.sub(r'\s+', ' ', minified)
        return f"{tag_open}{minified}</script>"
    
    html = re.sub(r'(<script[^>]*>)(.*?)</script>', minify_script_tag, html, flags=re.DOTALL | re.IGNORECASE)
    
    # 3. <pre>, <textarea> 태그 내부를 임시로 보호
    pre_blocks = []
    def save_pre(match):
        pre_blocks.append(match.group(0))
        return f"___PRE_BLOCK_{len(pre_blocks)-1}___"
    
    html = re.sub(r'<pre[^>]*>.*?</pre>', save_pre, html, flags=re.DOTALL | re.IGNORECASE)
    html = re.sub(r'<textarea[^>]*>.*?</textarea>', save_pre, html, flags=re.DOTALL | re.IGNORECASE)
    
    # 4. HTML 주석 제거
    html = re.sub(r'<!--.*?-->', '', html, flags=re.DOTALL)
    
    # 5. 태그 사이의 불필요한 공백 제거
    html = re.sub(r'>\s+<', '><', html)
    
    # 6. 연속된 공백을 하나로
    html = re.sub(r'\s+', ' ', html)
    
    # 7. 줄바꿈 제거
    html = html.replace('\n', '').replace('\r', '')
    
    # 8. 보호했던 pre/textarea 블록들 복원
    for i, block in enumerate(pre_blocks):
        html = html.replace(f"___PRE_BLOCK_{i}___", block)
    
    return html.strip()

def extract_doc_id(filename):
    """파일명에서 문서ID 추출 (예: 20151224_제목_2009465.html -> 2009465)"""
    # .html 제거
    name_without_ext = filename.replace('.html', '')
    # 마지막 언더스코어 뒤의 숫자 부분이 문서ID
    parts = name_without_ext.split('_')
    if len(parts) >= 2:
        # 마지막 부분이 숫자인지 확인
        last_part = parts[-1]
        if last_part.isdigit():
            return last_part
    return None

def modify_specific_css_rules(css_content):
    """특정 CSS 선택자의 속성만 수정"""
    
    # 1. .content_box 규칙 수정
    # .content_box{ ... } 블록을 찾아서 내부 속성 수정
    def modify_content_box(match):
        full_block = match.group(0)
        # min-width:850px → min-width:819px
        full_block = re.sub(r'min-width\s*:\s*850px', 'min-width:819px', full_block)
        # width:99.9%; 제거
        full_block = re.sub(r'width\s*:\s*99\.9%\s*;?', '', full_block)
        return full_block
    
    # .content_box{...} 블록 찾기 (중괄호 안의 내용 포함)
    css_content = re.sub(
        r'\.content_box\s*\{[^}]*\}',
        modify_content_box,
        css_content,
        flags=re.DOTALL
    )
    
    # 2. ul,ol 규칙 수정
    # ul,ol{list-style:none} → ul,ol{list-style:none !important; padding-left: 0 !important; margin-left: 0 !important;}
    def modify_ul_ol(match):
        # 기존 블록을 새로운 규칙으로 교체
        return 'ul,ol{list-style:none !important; padding-left: 0 !important; margin-left: 0 !important;}'
    
    # ul,ol{...} 블록 찾기
    css_content = re.sub(
        r'ul\s*,\s*ol\s*\{[^}]*\}',
        modify_ul_ol,
        css_content,
        flags=re.DOTALL
    )
    
    return css_content

def remove_html_wrapper_tags(html_content):
    """HTML wrapper 태그만 제거 (내용은 유지)"""
    
    # 정확한 태그만 한 줄씩 제거
    html_content = re.sub(r'<!DOCTYPE html>\s*', '', html_content, flags=re.IGNORECASE)
    html_content = re.sub(r'<html[^>]*>\s*', '', html_content, flags=re.IGNORECASE)
    html_content = re.sub(r'<head>\s*', '', html_content, flags=re.IGNORECASE)
    html_content = re.sub(r'</head>\s*', '', html_content, flags=re.IGNORECASE)
    html_content = re.sub(r'<body>\s*', '', html_content, flags=re.IGNORECASE)
    html_content = re.sub(r'</body>\s*', '', html_content, flags=re.IGNORECASE)
    html_content = re.sub(r'</html>\s*', '', html_content, flags=re.IGNORECASE)
    
    return html_content

def read_css_files(css_folder):
    """CSS 파일 1개를 읽어서 딕셔너리로 반환"""
    css_dict = {}
    css_files = [
        'apr.doc.print.jstl.css'  # 1개만 사용
    ]
    
    for css_file in css_files:
        css_path = css_folder / css_file
        if css_path.exists():
            with open(css_path, 'r', encoding='utf-8', errors='ignore') as f:
                css_content = f.read()
            
            # CSS 규칙 수정
            css_content = modify_specific_css_rules(css_content)
            css_dict[css_file] = css_content
            
            print(f"✓ CSS 파일 읽기 및 규칙 수정 완료: {css_file}")
        else:
            print(f"⚠️  CSS 파일을 찾을 수 없습니다: {css_file}")
    
    return css_dict

def insert_css_inline(html_content, css_dict):
    """HTML의 <link> 태그를 <style> 태그로 교체"""
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # 모든 <link> 태그 찾기
    link_tags = soup.find_all('link', rel='stylesheet')
    
    for link_tag in link_tags:
        href = link_tag.get('href', '')
        
        # CSS 파일명 추출
        css_filename = None
        if 'apr.doc.print.jstl.css' in href:
            css_filename = 'apr.doc.print.jstl.css'
        
        # 해당 CSS가 있으면 <style> 태그로 교체
        if css_filename and css_filename in css_dict:
            style_tag = soup.new_tag('style')
            style_tag.string = css_dict[css_filename]
            link_tag.replace_with(style_tag)
    
    return str(soup)

def remove_specific_sections(html_content):
    """첨부파일, 참조문서, 결재의견, 결재댓글 섹션 삭제"""
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # 1. 첨부파일 영역 삭제 (id="attachFileArea"인 div)
    for div in soup.find_all('div', id='attachFileArea'):
        div.decompose()
    
    # 2. 첨부파일 영역 삭제 (caption이 "파일,문서"인 경우)
    for div in soup.find_all('div', class_='table05_box'):
        caption = div.find('caption')
        if caption and '파일' in caption.get_text(strip=True):
            div.decompose()
    
    # 3. 첨부파일 행 삭제 (th에 "첨부파일" 텍스트가 있는 tr)
    for th in soup.find_all('th'):
        if '첨부파일' in th.get_text(strip=True):
            tr = th.find_parent('tr')
            if tr:
                tr.decompose()
    
    # 4. 참조문서 행 삭제 (th에 "참조문서" 텍스트가 있는 tr)
    for th in soup.find_all('th', scope='row'):
        if th.get_text(strip=True) == '참조문서':
            tr = th.find_parent('tr')
            if tr:
                tr.decompose()
    
    # 5. 결재의견 영역 삭제
    for div in soup.find_all('div', class_='table04_box'):
        caption = div.find('caption')
        if caption and '결재의견' in caption.get_text(strip=True):
            div.decompose()
    
    # 6. 결재댓글 영역 삭제
    for div in soup.find_all('div', class_='table04_box'):
        caption = div.find('caption')
        if caption and '결재댓글' in caption.get_text(strip=True):
            div.decompose()
    
    return str(soup)

def add_custom_css_to_styles(html_content):
    """모든 <style> 태그 맨 앞에 커스텀 CSS 추가"""
    soup = BeautifulSoup(html_content, 'html.parser')
    
    custom_css = '.content{width: 80% !important;} .content table {width: 100% !important;}'
    
    # 모든 <style> 태그 찾기
    style_tags = soup.find_all('style')
    
    for style_tag in style_tags:
        # 기존 내용 앞에 커스텀 CSS 추가
        original_content = style_tag.string if style_tag.string else ''
        style_tag.string = custom_css + original_content
    
    return str(soup)

def move_styles_before_body_close(html_content):
    """모든 <style> 태그를 </body> 태그 직전으로 이동"""
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # 모든 <style> 태그 찾기
    style_tags = soup.find_all('style')
    
    # body 태그 찾기
    body = soup.find('body')
    
    if body and style_tags:
        # 기존 위치에서 style 태그들을 제거하고 수집
        for style_tag in style_tags:
            style_tag.extract()
        
        # body 태그 끝에 추가
        for style_tag in style_tags:
            body.append(style_tag)
    
    return str(soup)

def check_and_fix_column_type(db_config):
    """doc_body 컬럼 타입 확인 및 필요시 변경"""
    conn = None
    cursor = None
    
    try:
        conn = pymysql.connect(
            host=db_config['host'],
            user=db_config['user'],
            password=db_config['password'],
            database=db_config['database'],
            charset='utf8mb4'
        )
        cursor = conn.cursor()
        
        # 컬럼 타입 확인
        cursor.execute("SHOW COLUMNS FROM documents LIKE 'doc_body'")
        result = cursor.fetchone()
        
        if result:
            column_type = result[1].decode('utf-8') if isinstance(result[1], bytes) else result[1]
            print(f"\n=== doc_body 컬럼 타입 확인 ===")
            print(f"현재 타입: {column_type}")
            
            # TEXT 타입이면 MEDIUMTEXT로 변경
            if column_type.upper() in ['TEXT', 'VARCHAR']:
                print(f"⚠️  {column_type}는 크기가 부족할 수 있습니다.")
                print(f"🔄 MEDIUMTEXT로 변경 중...")
                
                cursor.execute("ALTER TABLE documents MODIFY COLUMN doc_body MEDIUMTEXT")
                conn.commit()
                print(f"✅ doc_body 컬럼을 MEDIUMTEXT로 변경 완료!")
            elif column_type.upper() in ['MEDIUMTEXT', 'LONGTEXT']:
                print(f"✅ 컬럼 타입이 충분합니다.")
            else:
                print(f"⚠️  예상치 못한 타입입니다: {column_type}")
        
    except Exception as e:
        print(f"❌ 컬럼 타입 확인/변경 중 오류: {e}")
    
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

def update_db_docbody(processed_data, db_config, end_year):
    """처리된 HTML을 DB의 doc_body 컬럼에 업데이트"""
    conn = None
    cursor = None
    
    try:
        print("\n=== DB 연결 시작 ===")
        print(f"Host: {db_config['host']}, Database: {db_config['database']}")
        
        # DB 연결
        conn = pymysql.connect(
            host=db_config['host'],
            user=db_config['user'],
            password=db_config['password'],
            database=db_config['database'],
            charset='utf8mb4'
        )
        print("✓ DB 연결 성공")
        
        cursor = conn.cursor()
        
        # 업데이트 쿼리
        update_sql = """
        UPDATE documents 
        SET doc_body = %s 
        WHERE source_id = %s AND end_year = %s
        """
        
        success_count = 0
        fail_count = 0
        failed_items = []  # 실패한 항목 저장
        
        print(f"\n=== DB 업데이트 시작 ({len(processed_data)}건) ===")
        
        for idx, (doc_id, html_content) in enumerate(processed_data.items(), 1):
            try:
                html_size = len(html_content.encode('utf-8'))
                cursor.execute(update_sql, (html_content, doc_id, end_year))
                success_count += 1
                
                # 100건마다 출력
                if idx % 100 == 0 or idx == len(processed_data):
                    print(f"  진행: {idx}/{len(processed_data)} (성공: {success_count}, 실패: {fail_count})")
                    
            except Exception as e:
                fail_count += 1
                html_size = len(html_content.encode('utf-8'))
                failed_items.append({
                    'doc_id': doc_id,
                    'size': html_size,
                    'error': str(e)
                })
                
                if fail_count <= 10:
                    print(f"  [오류 {fail_count}] source_id: {doc_id}, 크기: {html_size:,} bytes - {str(e)[:80]}")
        
        # 커밋
        print("\n=== 커밋 시작 ===")
        conn.commit()
        print(f"✓ 커밋 완료")
        
        # 결과 요약
        print(f"\n=== DB 업데이트 결과 ===")
        print(f"✓ 성공: {success_count}건")
        print(f"✗ 실패: {fail_count}건")
        
        # 실패한 항목 상세 정보
        if failed_items:
            print(f"\n=== 실패 항목 분석 ===")
            sizes = [item['size'] for item in failed_items]
            print(f"실패한 항목 평균 크기: {sum(sizes)/len(sizes):,.0f} bytes")
            print(f"실패한 항목 최대 크기: {max(sizes):,} bytes")
            print(f"실패한 항목 최소 크기: {min(sizes):,} bytes")
            
            # 실패 항목을 파일로 저장
            with open('failed_updates.txt', 'w', encoding='utf-8') as f:
                for item in failed_items:
                    f.write(f"source_id: {item['doc_id']}, size: {item['size']:,} bytes, error: {item['error']}\n")
            print(f"✓ 실패 항목 상세 정보를 'failed_updates.txt'에 저장했습니다.")
        
    except pymysql.Error as e:
        print(f"\n❌ DB 오류 발생:")
        print(f"  Error Code: {e.args[0]}")
        print(f"  Error Message: {e.args[1]}")
        
    except Exception as e:
        print(f"\n❌ 일반 오류 발생: {e}")
        
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
            print("\n✓ DB 연결 종료")

def main():
    # 1. 경로 설정 (사용자 환경에 맞게 수정 필요)
    
    base_path = Path("C:/Users/LEEJUHWAN/Downloads/2010-01-01~2010-12-31/html") #여기를 수정하세요
    end_year = 2010  #여기를 수정하세요 - 폴더 기간에 맞게 수정
    approval_folder = base_path / "결재"
    css_folder = base_path / "resource" / "css"
    
    print(f"📂 기본 경로: {base_path}")
    print(f"📂 결재 폴더: {approval_folder}")
    print(f"📂 CSS 폴더: {css_folder}\n")
    
    # 경로 존재 확인
    if not approval_folder.exists():
        print(f"❌ 결재 폴더를 찾을 수 없습니다: {approval_folder}")
        print(f"💡 base_path 변수를 실제 경로로 수정해주세요!")
        return
    
    if not css_folder.exists():
        print(f"❌ CSS 폴더를 찾을 수 없습니다: {css_folder}")
        return
    
    # DB 설정
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': '1234',
        'database': 'any_approval' #여기를 수정하세요 - 통합 DB명
    }
    
    # DB 컬럼 타입 확인 및 필요시 변경
    check_and_fix_column_type(db_config)
    
    # 2. CSS 파일 읽기 및 규칙 수정
    print(f"\n🔄 CSS 파일 읽기 및 규칙 수정 중...\n")
    css_dict = read_css_files(css_folder)
    
    if not css_dict:
        print(f"❌ CSS 파일을 읽을 수 없습니다.")
        return
    
    print(f"\n✅ {len(css_dict)}개 CSS 파일 로드 및 수정 완료\n")
    
    # 3. 연도별 폴더에서 HTML 파일 수집
    print(f"🔍 HTML 파일 수집 중...\n")
    html_files = []
    
    for year_folder in sorted(approval_folder.iterdir()):
        if year_folder.is_dir():
            year = year_folder.name
            year_html_files = list(year_folder.glob("*.html"))
            html_files.extend(year_html_files)
            print(f"   📁 {year}년: {len(year_html_files)}개 파일")
    
    print(f"\n✅ 총 {len(html_files)}개 HTML 파일 발견\n")

    # html_files = html_files[:2]  # 테스트  #여기를 수정하세요
    
    # 4. HTML 파일 처리
    print(f"🔄 HTML 파일 처리 시작...\n")
    result_dict = {}
    success_count = 0
    fail_count = 0
    total_original_size = 0
    total_minified_size = 0
    
    for i, html_file in enumerate(html_files, 1):
        try:
            # 문서ID 추출
            doc_id = extract_doc_id(html_file.name)
            if not doc_id:
                print(f"⚠️  [{i}/{len(html_files)}] 문서ID 추출 실패: {html_file.name}")
                fail_count += 1
                continue
            
            # HTML 파일 읽기
            with open(html_file, 'r', encoding='utf-8', errors='ignore') as f:
                html_content = f.read()
            
            # 1. CSS inline 삽입
            html_with_css = insert_css_inline(html_content, css_dict)
            
            # 2. 특정 HTML 섹션 삭제
            html_cleaned = remove_specific_sections(html_with_css)
            
            # 3. 추가 CSS를 각 <style> 태그 맨 앞에 삽입
            html_with_custom_css = add_custom_css_to_styles(html_cleaned)
            
            # 4. 모든 <style> 태그를 </body> 앞으로 이동 (body 태그가 아직 있음)
            html_with_moved_styles = move_styles_before_body_close(html_with_custom_css)
            
            # 5. HTML wrapper 태그 제거 (이제 body 태그 삭제)
            html_without_wrapper = remove_html_wrapper_tags(html_with_moved_styles)
            
            # 6. HTML 압축
            minified_html = minify_html(html_without_wrapper)
            
            # 딕셔너리에 추가
            result_dict[doc_id] = minified_html
            
            success_count += 1
            
            # 압축 비율 계산
            original_size = len(html_without_wrapper)
            minified_size = len(minified_html)
            total_original_size += original_size
            total_minified_size += minified_size
            
            if i % 100 == 0 or i == len(html_files):
                print(f"✓ [{i}/{len(html_files)}] 진행 중... (성공: {success_count}, 실패: {fail_count})")
        
        except Exception as e:
            print(f"✗ [{i}/{len(html_files)}] 에러 발생: {html_file.name}")
            print(f"   에러 내용: {str(e)}")
            fail_count += 1
            continue
    
    # 5. DB 업데이트
    print(f"\n" + "="*60)
    update_db_docbody(result_dict, db_config, end_year)
    print(f"="*60)
    
    # 6. 결과 요약
    print(f"\n" + "="*60)
    print(f"✅ 완료!")
    print(f"="*60)
    print(f"📊 통계:")
    print(f"   - 전체 HTML 파일: {len(html_files)}개")
    print(f"   - 성공: {success_count}개")
    print(f"   - 실패: {fail_count}개")
    print(f"   - 총 원본 크기: {total_original_size:,} bytes")
    print(f"   - 총 압축 크기: {total_minified_size:,} bytes")
    if total_original_size > 0:
        total_ratio = (1 - total_minified_size/total_original_size) * 100
        print(f"   - 전체 압축률: ▼{total_ratio:.1f}%")

if __name__ == "__main__":
    main()


📂 기본 경로: C:\Users\LEEJUHWAN\Downloads\2010-01-01~2010-12-31\html
📂 결재 폴더: C:\Users\LEEJUHWAN\Downloads\2010-01-01~2010-12-31\html\결재
📂 CSS 폴더: C:\Users\LEEJUHWAN\Downloads\2010-01-01~2010-12-31\html\resource\css


=== doc_body 컬럼 타입 확인 ===
현재 타입: mediumtext
✅ 컬럼 타입이 충분합니다.

🔄 CSS 파일 읽기 및 규칙 수정 중...

✓ CSS 파일 읽기 및 규칙 수정 완료: apr.doc.print.jstl.css

✅ 1개 CSS 파일 로드 및 수정 완료

🔍 HTML 파일 수집 중...

   📁 2010년: 840개 파일

✅ 총 840개 HTML 파일 발견

🔄 HTML 파일 처리 시작...

✓ [100/840] 진행 중... (성공: 100, 실패: 0)
✓ [200/840] 진행 중... (성공: 200, 실패: 0)
✓ [300/840] 진행 중... (성공: 300, 실패: 0)
✓ [400/840] 진행 중... (성공: 400, 실패: 0)
✓ [500/840] 진행 중... (성공: 500, 실패: 0)
✓ [600/840] 진행 중... (성공: 600, 실패: 0)
✓ [700/840] 진행 중... (성공: 700, 실패: 0)
✓ [800/840] 진행 중... (성공: 800, 실패: 0)
✓ [840/840] 진행 중... (성공: 840, 실패: 0)


=== DB 연결 시작 ===
Host: localhost, Database: any_approval
✓ DB 연결 성공

=== DB 업데이트 시작 (840건) ===
  진행: 100/840 (성공: 100, 실패: 0)
  진행: 200/840 (성공: 200, 실패: 0)
  진행: 300/840 (성공: 300, 실패: 0)
  진행: 400/840 (성공: 400,